# EBSD/TKD analysis with kikuchipy and HyperSpy

## @ NNUM in Oslo, Norway June 4, 2024

Håkon Wiik Ånes (hwaanes@gmail.com), Xnovo Technology

In this tutorial we'll characterize an on-axis transmission Kikuchi diffraction (TKD) dataset of polycrystalline gold by Hough indexing and pattern matching.
We'll evaluate our results by using geometrical TKD simulations and comparing indexing results to pre-indexing maps.

The gold dataset is kindly provided by Alice Bastos da Silva from the Technical University of Denmark (DTU).
The patterns were acquired using Bruker Nano's on-axis TKD detector and software (Esprit).

Steps:

1. Load and inspect data
2. Specify candidate phases
3. Calibrate detector-sample geometry (projection/pattern center)
4. Hough indexing
5. Dictionary indexing
6. Orientation refinement

We link to tutorials relevant for each step in the respective sections in *italic*.

Documentation of tools we use:

* kikuchipy: https://kikuchipy.org
* HyperSpy: https://hyperspy.org
* PyEBSDIndex: https://pyebsdindex.readthedocs.io
* orix: https://orix.readthedocs.io
* diffsims: https://diffsims.readthedocs.io
* EMsoft (indirectly): https://github.com/EMsoft-org/EMsoft

Import all necessary functionality

In [ ]:
# Replace 'inline' for 'qt5' (from PyQt5 package) for interactive plotting
%matplotlib qt5
#%matplotlib inline

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from diffsims.crystallography import ReciprocalLatticeVector
import hyperspy.api as hs
from orix import io, plot, sampling
from orix.crystal_map import Phase, PhaseList
from orix.quaternion import Orientation, Rotation
from orix.vector import Vector3d

import kikuchipy as kp


savefig_kw = {"bbox_inches": "tight", "pad_inches": 0, "dpi": 300}

path_data = Path.cwd()
path_res = path_data / "results"
if not path_res.exists():
    path_res.mkdir()

## 1. Load and inspect data

*Relevant tutorial: https://kikuchipy.org/en/stable/tutorials/load_save_data.html.*

Load the 460 MB data into memory

In [ ]:
path_tkd = path_data / "20nmAu_2019-03-12_02.hdf5"

s = kp.load(path_tkd, lazy=False)

What is the shape and size of my EBSD data?

In [ ]:
s

In [ ]:
s.axes_manager

In [ ]:
s.data.shape

Note the flipping of dimensions here: HyperSpy uses (x, y, dx, dy), while NumPy stores the data as (y, x, dy, dx).

In [ ]:
s.data.dtype

What information about the experiment is available?

In [ ]:
s.metadata

In [ ]:
s.original_metadata

kikuchipy tries to read extra information specific to EBSD:
* Detector-sample calibration
* Static background (used to enhance the Kikuchi diffraction signal)
* Orientation data (indexing results)

In [ ]:
s.detector

In [ ]:
s.static_background

In [ ]:
s.xmap

In [ ]:
s.xmap = io.load(path_tkd)
s.xmap

In [ ]:
s.xmap.plot("RadonQuality")

We will produce our own indexing results and use most of the results from Bruker Esprit for comparison only.
The exception is the detector-sample calibration, which we will use as a starting guess for a refinement step.

### Some pre-indexing maps

*Relevant tutorials:*
* *https://kikuchipy.org/en/stable/tutorials/feature_maps.html*
* *https://kikuchipy.org/en/stable/tutorials/virtual_backscatter_electron_imaging.html*

Before indexing, it is important to:

* Evaluate the pattern quality
* Get an impression of what to expect from the indexing results

To that end, we inspect three different pre-indexing maps:

* Mean pattern intensity map
* Image quality map

Get the mean pattern intensity map by getting the average intensity within each pattern (axis 2 and 3 in our data array)

In [ ]:
s_mean = s.mean(axis=(2, 3))
plt.imsave(path_res / "maps_mean.png", s_mean.data, cmap="gray")
s_mean

Inspect patterns in this map

In [ ]:
s.plot(navigator=s_mean)

### Enhancement of Kikuchi pattern

*Relevant tutorial: https://kikuchipy.org/en/stable/tutorials/pattern_processing.html*.

Remove static (constant) background.
Instead of using the background acquired in the vendor software, we'll obtain one by averaging all patterns.
Before removing the static background, we compare the two static backgrounds.

In [ ]:
bg = s.mean(axis=(0, 1))
bg.metadata.General.title = "Our background"
bg.change_dtype(s.data.dtype)
bg

In [ ]:
bg2 = kp.signals.EBSD(s.static_background)
bg2.metadata.General.title = "Bruker's background"
print(bg2)
bg2.downsample(5)
bg2

In [ ]:
_ = hs.plot.plot_images([bg, bg2], axes_decor=None)

In [ ]:
s.remove_static_background(operation="divide", static_bg=bg.data)

Inspect a few (computed on the fly) statically corrected patterns

In [ ]:
s.plot(s_mean)

Save corrected patterns

In [ ]:
#s.save("patterns.h5")

### More pre-indexing maps

Get image quality $Q$ map (not image quality IQ from Hough indexing!)

In [ ]:
maps_iq = s.get_image_quality()
plt.imsave(path_res / "maps_iq.png", maps_iq, cmap="gray")

Navigate patterns in $Q$ map

In [ ]:
s.plot(hs.signals.Signal2D(maps_iq), vmin=10_000)

## 2. Specify candidate phases

*Relevant tutorial: https://kikuchipy.org/en/stable/tutorials/kinematical_ebsd_simulations.html.*

We do not expect other phases than nickel.
We will load the master pattern of nickel (packaged with kikuchipy) created with EMsoft

In [ ]:
path_mp = path_data / "au_mc_mp_tkd_30kv.h5"

mp = kp.load(path_mp, energy=30, projection="lambert")

mp.phase.name = "au"
mp

Create a phase list for use in Hough indexing, and set the lattice parameters to Ångström

In [ ]:
pl = PhaseList(mp.phase)
lat = pl["au"].structure.lattice
lat.setLatPar(lat.a * 10, lat.b * 10, lat.c * 10)

pl

Also, load the stereographic projection which we will plot later to understand a bit more about simulations

In [ ]:
mp_sp = kp.load(path_mp, energy=30, projection="stereographic")

If we have PyVista (and VTK) installed, we can inspect the Kikuchi sphere in 3D

In [ ]:
#mp_sp.plot_spherical(style="points", plotter_kwargs={"notebook": False})

Otherwise, we'll have to live with the 2D stereographic projection

In [ ]:
mp_sp.plot()

## 3. Calibrate detector-sample geometry

*Relevant tutorials:*
* *https://kikuchipy.org/en/stable/tutorials/reference_frames.html*
* *https://kikuchipy.org/en/stable/tutorials/pc_extrapolate_plane.html*
* *https://kikuchipy.org/en/stable/tutorials/pc_fit_plane.html*
* *https://kikuchipy.org/en/stable/tutorials/pc_calibration_moving_screen_technique.html*
* *https://kikuchipy.org/en/stable/tutorials/pc_orientation_dependence.html*

kikuchipy's reference frames (taken from the documentation):

<img src="https://kikuchipy.org/en/stable/_images/sample_detector_geometry.png" width="700">

Inspect calibration read from the vendor

In [ ]:
s.detector.sample_tilt = 90.0
s.detector

Orientation of detector with respect to the sample:
* Known (fixed):
    * Sample tilt $\sigma$ = 90.0 degrees
    * Camera tilt $\theta$ = 5.83 degrees
* Unknown (but with a good guess):
    * Projection center (PCx, PCy, PCz): Shortest distance from source point to detector

Find good-quality calibration patterns to get a mean PC for the data

In [ ]:
s.plot(hs.signals.Signal2D(maps_iq))

In [ ]:
xy = [
    (14, 0),
    (10, 12),
    (17, 32),
    (15, 53),
    (37, 51),
    (42, 37),
    (42, 22),
]
p_cal = np.zeros((len(xy),) + s.detector.shape, dtype=s.data.dtype)
for i, (x, y) in enumerate(xy):
    p_cal[i] = s.data[y, x]
s_cal = kp.signals.EBSD(p_cal, detector=s.detector)

for i, name in zip(range(3), ["x", "dx", "dy"]):
    s_cal.axes_manager[i].name = name

Plot pattern positions in an overview image (specific to NORDIF acquisition software)

In [ ]:
rc = np.array(xy)[:, ::-1]

fig = kp.draw.plot_pattern_positions_in_map(
    rc=rc,
    roi_shape=maps_iq.shape,
    roi_image=maps_iq,
    color="r",
    return_figure=True,
)
fig.savefig(path_res / "calibration_pattern_positions.png", **savefig_kw)

Inspect calibration patterns

In [ ]:
s_cal.plot(None)

Now we're ready to make validate our calibration (estimates of the projection centers, PCs).
We do this by optimizing the PCs and use these to Hough index our calibration patterns.

Hough indexing requires us to select a few strongly scattering reflections.
We select the four typically brighest reflections for gold

In [ ]:
g = ReciprocalLatticeVector(pl["au"], hkl=[[1, 1, 1], [2, 0, 0], [2, 2, 0], [3, 1, 1]])
g = g.symmetrise()
g.sanitise_phase()  # Complete unit cell
g.calculate_structure_factor()
g.calculate_theta(30e3)
g.print_table()

Create a simulator to inspect the reflectors on the Kikuchi sphere (and for later simulations)

In [ ]:
simulator = kp.simulations.KikuchiPatternSimulator(g)

Plot simulator

In [ ]:
simulator.plot(mode="bands", axes_labels=["e1", "e2"])

Generate an indexer for PC optimization and Hough indexing (required by PyEBSDIndex)

In [ ]:
indexer_cal = s_cal.detector.get_indexer(pl, g)

Optimize PC by trial and error (the initial guess is based on previous experiments on the same microscope)

In [ ]:
s.detector

In [ ]:
det_cal = s_cal.hough_indexing_optimize_pc([0.49, 0.44, -0.63], indexer_cal, batch=True, method="PSO")

# Print PCs and standard deviations
print("Mean PC:\n", det_cal.pc_average)
print("Std:\n", det_cal.pc.std(axis=0), "\n")
print("All PCs:\n", det_cal.pc)

# Get new indexer from detector, using the average PC
indexer_cal2 = det_cal.get_indexer(pl)

# Index calibration patterns using found PCs
xmap_cal = s_cal.hough_indexing(pl, indexer_cal2, verbose=2)
print("\nAll indexed:", xmap_cal.is_indexed.all(), "\n")

# Create geometrical simulations for each pattern
sim_cal = simulator.on_detector(det_cal, xmap_cal.rotations)

Plot patterns with markers

In [ ]:
#del s_cal.metadata.Markers  # Uncomment to delete previous markers
markers = sim_cal.as_markers(lines_kwargs={"linewidth": 0.5, "color": "r"})
s_cal.add_marker(markers, plot_marker=False, permanent=True)
s_cal.plot(None, vmin=10_000)

Plot pair of PC coordinates and compare to the map of PC positions from above

In [ ]:
det_cal.plot_pc("scatter", annotate=True)

Refine PCs using pattern matching

In [ ]:
xmap_cal2, det_cal2 = s_cal.refine_orientation_projection_center(
    xmap_cal,
    det_cal,
    mp,
    energy=30,
    method="LN_NELDERMEAD",  # Comment to use SciPy instead
    trust_region=[5, 5, 5, 0.1, 0.1, 0.1],
    chunk_kwargs={"chunk_shape": 1},  # One pattern per CPU
)

# Create geometrical simulations for each pattern
sim_cal2 = simulator.on_detector(det_cal2, xmap_cal2.rotations)

Again, plot patterns with markers

In [ ]:
markers2 = sim_cal2.as_markers(lines_kwargs={"linewidth": 0.5, "color": "w"})
s_cal.add_marker(markers2, plot_marker=False, permanent=True)

In [ ]:
s_cal.plot(None, vmin=10_000)

We can also compare dynamically simulated equivalents of the experimental patterns

In [ ]:
sim_cal = mp.get_patterns(xmap_cal2.rotations, det_cal2, compute=True)
sim_cal

Remove the line markers and stretch intensities for the best visual comparison

In [ ]:
del s_cal.metadata.Markers

s_cal2 = s_cal.deepcopy()
s_cal2.normalize_intensity(dtype_out="float32")
sim_cal2 = sim_cal.deepcopy()
sim_cal2.normalize_intensity(dtype_out="float32")

In [ ]:
hs.plot.plot_signals([s_cal2, sim_cal2], navigator=None, vmin=-3, vmax=3)

In [ ]:
det_cal2.plot_pc("scatter", annotate=True)

Save detector

In [ ]:
det_cal2.save(path_res / "det_cal.txt")

When the map covers a large area on the sample, the PC moves quite a lot on the detector.
This is not the case here, as our map covers about x$ x $y$ = 305 x 255 um^2.
It is therefore valid to use a single PC for the whole map, which we will do in the following.

In [ ]:
det1 = det_cal2.deepcopy()
det1.pc = det1.pc_average

## 4. Hough indexing

*Relevant tutorials:*
* *https://kikuchipy.org/en/stable/tutorials/hough_indexing.html#*
* *https://kikuchipy.org/en/stable/tutorials/hybrid_indexing.html*
* *https://pyebsdindex.readthedocs.io/en/latest/tutorials/ebsd_index_demo.html*

Create a new indexer with the single PC

In [ ]:
indexer = det1.get_indexer(pl, g)

Hough index all patterns

In [ ]:
xmap_hi = s.hough_indexing(pl, indexer, verbose=2)

Check whether any patterns could not be indexed

In [ ]:
xmap_hi

Save the Hough indexing results

In [ ]:
io.save(path_res / "xmap_hi.h5", xmap_hi, overwrite=True)

Plot the indexing quality metrics per pattern

In [ ]:
aspect_ratio = xmap_hi.shape[1] / xmap_hi.shape[0]
figsize = (10 * aspect_ratio, 10 * aspect_ratio)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=figsize, layout="tight")
for ax, to_plot in zip(axes.ravel(), ["pq", "cm", "fit", "nmatch"]):
    im = ax.imshow(xmap_hi["indexed"].get_map_data(to_plot))
    fig.colorbar(im, ax=ax, label=to_plot)
    ax.axis("off")

Histograms

In [ ]:
fig, axes = plt.subplots(nrows=4, figsize=(4, 10), layout="tight")
for ax, to_plot in zip(axes.ravel(), ["pq", "cm", "fit", "nmatch"]):
    _ = ax.hist(xmap_hi["indexed"].prop[to_plot], bins=100)
    ax.set(xlabel=to_plot, ylabel="Frequency")
    if to_plot == "pq":
        ax.ticklabel_format(axis="x", style="sci", scilimits=(0, 0))

Plot the inverse pole figure IPF-Z map, where colors are given by the (symmetry reduced) crystal direction $\mathbf{t} = [uvw]$ pointing into-plane

In [ ]:
ckey = plot.IPFColorKeyTSL(xmap_hi.phases[0].point_group, direction=Vector3d([0, 0, 1.]))

In [ ]:
ckey.plot()

In [ ]:
rgb_hi = ckey.orientation2color(xmap_hi.rotations)
fig_hi = xmap_hi.plot(rgb_hi, return_figure=True)
fig_hi.axes[0].set_title("PyEBSDIndex");

Plot the IPF-Z map overlayed with the confidence metric

In [ ]:
xmap_hi.plot(rgb_hi, overlay="cm")

Compare to the results from Bruker's software

In [ ]:
R_bruker = s.xmap.rotations
R_bruker2kp = Rotation.from_axes_angles([0, 0, 1], 90, degrees=True)

rgb_bruker = ckey.orientation2color(R_bruker * R_bruker2kp)
fig_bruker = s.xmap.plot(rgb_bruker, return_figure=True)
fig_bruker.axes[0].set_title("Bruker");

Evaluate results by plotting geometrical simulations on top of patterns

In [ ]:
# Temporary fix for a bug!
s.axes_manager["dx"].scale = s.axes_manager["dy"].scale = 1.

In [ ]:
sim_hi = simulator.on_detector(det1, xmap_hi.rotations.reshape(*xmap_hi.shape))

In [ ]:
#del s.metadata.Markers  # Uncomment to delete previously added markers
markers = sim_hi.as_markers(lines_kwargs={"linewidth": 0.5})
s.add_marker(markers, permanent=True, plot_marker=False)

Create a navigator from the IPF-Z map

In [ ]:
maps_ipf = kp.draw.get_rgb_navigator(rgb_hi.reshape(xmap_hi.shape + (3,)))

In [ ]:
s.plot(maps_ipf)

## 5. Dictionary indexing

Dictionary indexing is more robust towards noise in EBSD patterns (resulting from e.g. overlapping bands, high degree of deformation etc.) than Hough indexing.
A dictionary consists of a series of (dynamically) simulated patterns projected from a master pattern, and we compare all experimental patterns to all these.

To create this dictionary, we need:

* Accurate detector-sample geometry (already done!)
* Master pattern (in the square Lambert projection)
* Sampling of all possible orientations (per phase)

Plot our geometrical simulation on top of the upper stereographic projection of our dynamically simulated Ni Kikuchi sphere

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "stereographic"}, figsize=(7, 7))
simulator.plot(mode="bands", color="w", figure=fig)
ax.imshow(mp_sp.data, cmap="gray", extent=(-1, 1, -1, 1));

Discretely sample the complete orientation space of point group $m\bar{3}m$ (*Oh*) with an average misorientation of about 2$^{\circ}$ between rotations $\mathbf{g}$

In [ ]:
R_sample = sampling.get_sample_fundamental(
    resolution=2, point_group=mp.phase.point_group
)
O_sample = Orientation(R_sample, symmetry=mp.phase.point_group)

In [ ]:
O_sample

Plot a subset of sampled orientations in axis-angle space

In [ ]:
O_sample.get_random_sample(2000).scatter()

Bin patterns

In [ ]:
s2 = s.downsample(2, inplace=False)
s2

In [ ]:
det2 = det1.deepcopy()
det2.shape = s2.axes_manager.signal_shape[::-1]

Set up generation of the dictionary of dynamically simulated patterns

In [ ]:
s_dict = mp.get_patterns(R_sample, det2, energy=30, chunk_shape=1000)
s_dict

Inspect the five first patterns in the dictionary

In [ ]:
fig = plt.figure(figsize=(20, 4), layout="tight")
hs.plot.plot_images(s_dict.inav[:5], axes_decor=None, per_row=5, fig=fig, colorbar=False);

Perform dictionary indexing by generating a chunk of simulated patterns at a time and compare them to all the experimental patterns

In [ ]:
xmap_di = s2.dictionary_indexing(s_dict)

In [ ]:
xmap_di

The 20 best matches (rotations, scores and simulation indices) are kept

In [ ]:
xmap_di.scores.shape

Plot similarity scores (normalized cross-correlation, NCC) between best matching
experimental and simulated patterns

In [ ]:
xmap_di.plot(xmap_di.scores[:, 0], colorbar=True, colorbar_label="NCC")

Plot IPF-Z orientation map

In [ ]:
rgb_di = ckey.orientation2color(xmap_di.rotations[:, 0])
xmap_di.plot(rgb_di)

Plot IPF-Z orientation map with scores

In [ ]:
xmap_di.plot(rgb_di, overlay=xmap_di.scores[:, 0])

Save dictionary indexing results to file

In [ ]:
io.save(path_res / "xmap_di.h5", xmap_di, overwrite=True)

## 6. Orientation refinement

During refinement, a better score $r$ is searched for iteratively by changing the orientation (and/or PC) slightly in a controlled manner using an optimization algorithm.
The default algorithm is the Nelder-Mead simplex from SciPy.
We here use that from the NLopt package (an optional dependency of kikuchipy), which has been found to be faster but equally accurate.

In [ ]:
xmap_ref = s.refine_orientation(
    xmap=xmap_di,
    detector=det1,
    master_pattern=mp,
    energy=30,
    method="LN_NELDERMEAD",  # Comment to use SciPy instead
    trust_region=[2, 2, 2],
    rechunk=True,
)

In [ ]:
xmap_ref

Plot refined orientation map

In [ ]:
rgb_ref = ckey.orientation2color(xmap_ref.rotations)
xmap_ref.plot(rgb_ref)

Plot map with refined NCC scores

In [ ]:
xmap_ref.plot(xmap_ref.scores, colorbar=True, colorbar_label="NCC")

Plot orientation map with scores overlayed

In [ ]:
xmap_ref.plot(rgb_ref, overlay="scores")

Compare histogram of scores after DI and refinement

In [ ]:
fig, ax = plt.subplots()
ax.hist(xmap_di.scores[:, 0], bins=100, color="C0", label="DI", alpha=0.5)
ax.hist(xmap_ref.scores, bins=100, color="C1", label="Ref", alpha=0.5)
ax.legend()
ax.set(xlabel="NCC", ylabel="Frequency");

Save final indexing results

In [ ]:
io.save(path_res / "xmap_ref.ang", xmap_ref, overwrite=True)
io.save(path_res / "xmap_ref.h5", xmap_ref, overwrite=True)

Compare to simulations

In [ ]:
sim_ref = simulator.on_detector(det1, xmap_ref.rotations.reshape(*xmap_ref.shape))

In [ ]:
markers = sim_ref.as_markers(lines_kwargs={"linewidth": 0.5})

In [ ]:
del s.metadata.Markers
s.add_marker(markers, permanent=True, plot_marker=False)

Create a navigator from the IPF-Z map

In [ ]:
maps_ipf = kp.draw.get_rgb_navigator(rgb_ref.reshape(xmap_ref.shape + (3,)))

In [ ]:
s.plot(maps_ipf)

Compare IPF maps from all four indexing routes

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(6, 8), layout="tight")
for ax, rgb_i, title in zip(
    axes.ravel(),
    [rgb_bruker, rgb_hi, rgb_di, rgb_ref],
    ["Bruker", "PyEBSDIndex", "DI + ref.", "DI + ref."]
):
    ax.imshow(rgb_i.reshape(xmap_ref.shape + (3,)))
    ax.set_title(title)
    ax.axis("off")
fig.savefig(path_res / "maps_ipf001.png", **savefig_kw)

## What next?

We have now indexed our patterns and validated the results.
A natural next step in the analysis of the orientation data may be to analyze the grain size and morphology, as well as their preferred orientation (texture).
The results produced here can be imported by e.g. [MTEX](https://mtex-toolbox.github.io/) or [DREAM.3D](https://www.dream3d.io/).
Happy orientation data analysis!